In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import pandas as pd
import subprocess
import json
import os  # Import the os module
from moviepy.editor import VideoFileClip, concatenate_videoclips

# # Base path to the course folder
# root_dir = '/content/drive/MyDrive/MASTERMIDJOURNEYOCT24/MASTERMIDJOURNEYOCT24_LESSON1/'  # Update to match the base path
# lesson_number = 3  # Update the lesson number dynamically

# Загрузка таблицы с разметкой
# Загрузка таблицы с разметкой
segments_df = pd.read_csv('/content/drive/MyDrive/MIDJOUROCT24/MASTERMIDJOURNEYOCT24_LESSON1/A1nightly2.csv')

# Определение параметров видео
input_video_path = '/content/drive/MyDrive/MIDJOUROCT24/MASTERMIDJOURNEYOCT24_LESSON1/LESSON1.mp4'
output_folder_path = '/content/drive/MyDrive/MIDJOUROCT24/MASTERMIDJOURNEYOCT24_LESSON1/Videofragments2/'

# Функция для получения частоты кадров видео
def get_frame_rate(input_file):
    cmd = ['ffprobe', '-v', 'error', '-select_streams', 'v:0', '-count_packets',
           '-show_entries', 'stream=r_frame_rate', '-of', 'json', input_file]
    result = subprocess.run(cmd, capture_output=True, text=True)
    data = json.loads(result.stdout)
    fps_str = data['streams'][0]['r_frame_rate']
    num, den = map(int, fps_str.split('/'))
    return num / den

# Получение частоты кадров из входного видео
fps = get_frame_rate(input_video_path)

# Открываем входное видео
video_clip = VideoFileClip(input_video_path)

# Группировка сегментов по section_name
grouped_segments = segments_df.groupby('section_name')

# Функция для создания клипов по уникальным section_name
for section_name, group in grouped_segments:
    merged_segments = []
    current_start = None
    current_end = None

    for i, row in group.iterrows():
        segment_start = row['sentence_start']
        segment_end = row['sentence_end']

        if current_start is None:
            current_start = segment_start
            current_end = segment_end
        else:
            if segment_start - current_end <= 1.5:
                # Расстояние меньше 1.5 секунд, объединяем сегменты
                current_end = segment_end
            else:
                # Добавляем предыдущий объединенный сегмент
                merged_segments.append((current_start, current_end))
                # Начинаем новый сегмент
                current_start = segment_start
                current_end = segment_end

     # Добавляем последний сегмент
    if current_start is not None:
        merged_segments.append((current_start, current_end))

        section_name_sanitized = section_name.replace(' ', '_')  # Replace spaces with underscores
        output_clip_path = os.path.join(output_folder_path, f"{section_name_sanitized}.mp4")


    # Создание большого видео для текущего section_name
    section_clips = []
    for start, end in merged_segments:
        # Создание отрезка видео с учетом fps
        clip = video_clip.subclip(start, end).set_fps(fps)
        section_clips.append(clip)

    # Объединение всех клипов в один для текущего section_name
    if section_clips:
        final_clip = concatenate_videoclips(section_clips)
        output_clip_path = f"{output_folder_path}{section_name.replace(' ', '_')}.mp4"

        # Использование FFmpeg для сохранения объединенного клипа с точными параметрами
        temp_instruction_path = 'temp_instructions.txt'
        with open(temp_instruction_path, 'w') as f:
            for start, end in merged_segments:
                f.write(f"file '{input_video_path}'\n")
                f.write(f"inpoint {start:.6f}\n")
                f.write(f"outpoint {end:.6f}\n")

        # FFmpeg командная строка для точного обрезания и объединения
        command = [
            'ffmpeg',
            '-y',  # Add this flag to overwrite output files without asking
            '-f', 'concat',
            '-safe', '0',
            '-i', temp_instruction_path,
            '-vsync', '0',  # Maintain original timestamps
            '-af', 'aresample=async=1',  # Light audio resampling for sync
            '-c:v', 'libx264',
            '-preset', 'slow',
            '-crf', '18',
            '-c:a', 'aac',
            '-b:a', '192k',
            '-movflags', '+faststart',
            output_clip_path
        ]
        try:
            result = subprocess.run(command, check=True, capture_output=True, text=True)
            print("FFmpeg command executed successfully")
        except subprocess.CalledProcessError as e:
            print(f"FFmpeg command failed with error:\n{e.stderr}")
             # Print the FFmpeg command for debugging
            print(f"FFmpeg command: {' '.join(command)}")
            raise e
        finally:
            # Clean up the temporary instructions file
            import os
            os.remove(temp_instruction_path)

# Закрываем входной клип
video_clip.close()




FFmpeg command executed successfully
FFmpeg command executed successfully
FFmpeg command executed successfully
FFmpeg command executed successfully
FFmpeg command executed successfully
FFmpeg command executed successfully
FFmpeg command executed successfully
FFmpeg command executed successfully
FFmpeg command executed successfully
FFmpeg command executed successfully
FFmpeg command executed successfully


In [11]:
# prompt: How do I also ask to print names of the videos into new df in the root folder, which will collect all names of generated videos and also will include the number of the lesson?

from google.colab import drive
from google.colab import sheets
import pandas as pd
import subprocess
import json
import os  # Import the os module
from moviepy.editor import VideoFileClip, concatenate_videoclips


# # Base path to the course folder
# root_dir = '/content/drive/MyDrive/MASTERMIDJOURNEYOCT24/MASTERMIDJOURNEYOCT24_LESSON1/'  # Update to match the base path
# lesson_number = 3  # Update the lesson number dynamically

# Загрузка таблицы с разметкой
segments_df = pd.read_csv('/content/drive/MyDrive/MIDJOUROCT24/MASTERMIDJOURNEYOCT24_LESSON1/A1nightly2.csv')

# Определение параметров видео
input_video_path = '/content/drive/MyDrive/MIDJOUROCT24/MASTERMIDJOURNEYOCT24_LESSON1/LESSON1.mp4'
output_folder_path = '/content/drive/MyDrive/MASTERMIDJOURNEYOCT24/MASTERMIDJOURNEYOCT24_LESSON1/Videofragments2/'

# Функция для получения частоты кадров видео
def get_frame_rate(input_file):
    cmd = ['ffprobe', '-v', 'error', '-select_streams', 'v:0', '-count_packets',
           '-show_entries', 'stream=r_frame_rate', '-of', 'json', input_file]
    result = subprocess.run(cmd, capture_output=True, text=True)
    data = json.loads(result.stdout)
    fps_str = data['streams'][0]['r_frame_rate']
    num, den = map(int, fps_str.split('/'))
    return num / den

# Получение частоты кадров из входного видео
fps = get_frame_rate(input_video_path)

# Открываем входное видео
video_clip = VideoFileClip(input_video_path)

# Группировка сегментов по section_name
grouped_segments = segments_df.groupby('section_name')

# Инициализация DataFrame для хранения имен видео
video_names_df = pd.DataFrame(columns=['Lesson', 'Video Name'])
lesson_number = 1  # Укажите номер урока

# Функция для создания клипов по уникальным section_name
for section_name, group in grouped_segments:
    merged_segments = []
    current_start = None
    current_end = None

    for i, row in group.iterrows():
        segment_start = row['sentence_start']
        segment_end = row['sentence_end']

        if current_start is None:
            current_start = segment_start
            current_end = segment_end
        else:
            if segment_start - current_end <= 1.5:
                # Расстояние меньше 1.5 секунд, объединяем сегменты
                current_end = segment_end
            else:
                # Добавляем предыдущий объединенный сегмент
                merged_segments.append((current_start, current_end))
                # Начинаем новый сегмент
                current_start = segment_start
                current_end = segment_end

     # Добавляем последний сегмент
    if current_start is not None:
        merged_segments.append((current_start, current_end))

        section_name_sanitized = section_name.replace(' ', '_')  # Replace spaces with underscores
        output_clip_path = os.path.join(output_folder_path, f"{section_name_sanitized}.mp4")


    # Создание большого видео для текущего section_name
    section_clips = []
    for start, end in merged_segments:
        # Создание отрезка видео с учетом fps
        clip = video_clip.subclip(start, end).set_fps(fps)
        section_clips.append(clip)

    # Объединение всех клипов в один для текущего section_name
    if section_clips:
        final_clip = concatenate_videoclips(section_clips)
        output_clip_path = f"{output_folder_path}{section_name.replace(' ', '_')}.mp4"

        # Использование FFmpeg для сохранения объединенного клипа с точными параметрами
        temp_instruction_path = 'temp_instructions.txt'
        with open(temp_instruction_path, 'w') as f:
            for start, end in merged_segments:
                f.write(f"file '{input_video_path}'\n")
                f.write(f"inpoint {start:.6f}\n")
                f.write(f"outpoint {end:.6f}\n")

        # FFmpeg командная строка для точного обрезания и объединения
        command = [
            'ffmpeg',
            '-y',  # Add this flag to overwrite output files without asking
            '-f', 'concat',
            '-safe', '0',
            '-i', temp_instruction_path,
            '-vsync', '0',  # Maintain original timestamps
            '-af', 'aresample=async=1',  # Light audio resampling for sync
            '-c:v', 'libx264',
            '-preset', 'slow',
            '-crf', '18',
            '-c:a', 'aac',
            '-b:a', '192k',
            '-movflags', '+faststart',
            output_clip_path
        ]
        try:
            result = subprocess.run(command, check=True, capture_output=True, text=True)
            print("FFmpeg command executed successfully")
            # Добавление имени видео в DataFrame
            video_names_df = pd.concat([video_names_df, pd.DataFrame({'Lesson': [lesson_number], 'Video Name': [section_name.replace(' ', '_') + '.mp4']})], ignore_index=True)
        except subprocess.CalledProcessError as e:
            print(f"FFmpeg command failed with error:\n{e.stderr}")
             # Print the FFmpeg command for debugging
            print(f"FFmpeg command: {' '.join(command)}")
            raise e
        finally:
            # Clean up the temporary instructions file
            os.remove(temp_instruction_path)


# Закрываем входной клип
video_clip.close()

# Сохранение DataFrame с именами видео
video_names_df.to_csv(f'(root_dir)/video_names.csv', index=False)
print("DataFrame с именами видео сохранен в root folder")


FFmpeg command failed with error:
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enabl

CalledProcessError: Command '['ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', 'temp_instructions.txt', '-vsync', '0', '-af', 'aresample=async=1', '-c:v', 'libx264', '-preset', 'slow', '-crf', '18', '-c:a', 'aac', '-b:a', '192k', '-movflags', '+faststart', '/content/drive/MyDrive/MASTERMIDJOURNEYOCT24/MASTERMIDJOURNEYOCT24_LESSON1/Videofragments2/1.Greetings_and_Check-in:_What_do_you_already_know_about_the_most_beautiful_diffusion_model_in_Human_history.mp4']' returned non-zero exit status 1.